In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import h5py
from PIL import Image
import scipy 
from scipy import ndimage
from lr_utils import load_dataset
import skimage
%matplotlib inline


ModuleNotFoundError: No module named 'lr_utils'

In [ ]:
train_set_x_orig,train_set_y,test_set_x_orig,test_set_y,classes=load_dataset()

In [ ]:
index=25
example = train_set_x_orig[index]
plt.imshow(example)

In [ ]:
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[2]
print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

In [ ]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T

In [ ]:
print("shape of trianing x set"+str(train_set_x_flatten.shape))
print("shape of training y set"+str(train_set_y.shape))
print("shape of testing x set"+str(test_set_x_flatten.shape))
print("shape of testing y set"+ str(test_set_y.shape))

In [ ]:
train_set_x_flatten = train_set_x_flatten/255


In [ ]:
test_set_x_flatten = test_set_x_flatten/255

In [ ]:
def sigmoid(z):
    s = 1./(1+np.exp(-z))
    return s

In [ ]:
def initialization(dim):
    w = np.zeros(shape=(dim,1),dtype = np.float32)
    b = 0
    assert(w.shape==(dim,1))
    assert(isinstance(b,float) or isinstance(b,int))
    return w,b

In [ ]:
def propagate(w,b,X,Y):
    m=X.shape[0]
    A = sigmoid(np.dot(w.T,X)+b)
    cost = (-1./m)*np.sum((Y*np.log(A) + (1-Y)*np.log(1-A)),axis = 1)
    dw = (1./m)*np.dot(X,(A-Y).T)
    db = (1./m)*np.sum(A-Y,axis=1)
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape==())
    
    
    grads = {"dw":dw,
             "db":db
            }
    
    return grads,cost

    
    

In [ ]:
def optimize(w,b,X,Y,num_iterations,learning_rate,print_cost = False):
    costs=[]
    for i in range(num_iterations):
        grads,cost = propagate(w=w,b=b,X=X,Y=Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate*dw
        b = b - learning_rate*db
        
        if i%100 == 0:
            costs.append(cost)
    
        if print_cost and i%100 == 0:
            print("cost after iteration %i: %f:" %(i,cost))
        
    grads = {"dw":dw,
             "db":db}
    params = {"w":w,
             "b":b}
    return params,grads,cost


In [ ]:
def predict(w,b,X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0],1)
    A = sigmoid(np.dot(w.T,X)+b)
    
    [print(x) for x in A]
    for i in range(A.shape[1]):
        if A[0, i] >= 0.5:
            Y_prediction[0, i] = 1
            
        else:
            Y_prediction[0, i] = 0
    assert(Y_prediction.shape == (1,m))
    return Y_prediction

    
    

In [ ]:
def model(X_train,Y_train,X_test,Y_test,num_iterations=2000,learning_rate=0.5,print_cost=False):
    w,b = initialization(X_train.shape[0])
    parameters, grads,cost = optimize(w=w,b=b,X=X_train,Y=Y_train,num_iterations=num_iterations,learning_rate=learning_rate,print_cost=print_cost)
    w = parameters["w"]
    b = parameters["b"]
    
    
    Y_prediction_test = predict(w,b,X_test)
    Y_prediction_train = predict(w,b,X_train)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    
    d = {"costs": cost,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

    
    

In [ ]:
d = model(train_set_x_flatten,train_set_y,test_set_x_flatten,test_set_y,num_iterations=2000,learning_rate=0.05,print_cost=True)

In [ ]:
index = 1
plt.imshow(test_set_x_flatten[:,index].reshape((num_px, num_px, 3)))
print ("y = " + str(test_set_y[0,index]) + ", you predicted that it is a \"" + classes[int(d["Y_prediction_test"][0,index])].decode("utf-8") +  "\" picture.")